In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif, SelectKBest
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('HR_Attrition_dataset.csv')

In [4]:
df.head(2)

,Employee_ID,Age,Attrition,Business_Travel,Department,Distance_From_Home,Education,Environment_Satisfaction,Gender,Salary,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Number_of_Companies_Worked_previously,Overtime,Salary_Hike_in_percent,Total_working_years_experience,Work_life_balance,No_of_years_worked_at_current_company,No_of_years_in_current_role,Years_since_last_promotion
0,1,56,Yes,Travel Rarely,Network Administration,24,Graduation,5,Male,186630,2,6,Manager,4,Married,7,Yes,85,9,1,7,2,4
1,2,46,Yes,Travel Rarely,Software Development,45,Degree,5,Male,78553,4,5,Help Desk,2,Divorced,3,No,22,18,4,1,2,0


In [5]:
df['Attrition'] = df['Attrition'].map({'Yes':1, 'No':0})

In [6]:
df.head(1)

,Employee_ID,Age,Attrition,Business_Travel,Department,Distance_From_Home,Education,Environment_Satisfaction,Gender,Salary,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Number_of_Companies_Worked_previously,Overtime,Salary_Hike_in_percent,Total_working_years_experience,Work_life_balance,No_of_years_worked_at_current_company,No_of_years_in_current_role,Years_since_last_promotion
0,1,56,1,Travel Rarely,Network Administration,24,Graduation,5,Male,186630,2,6,Manager,4,Married,7,Yes,85,9,1,7,2,4


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Employee_ID                            10000 non-null  int64 
 1   Age                                    10000 non-null  int64 
 2   Attrition                              10000 non-null  int64 
 3   Business_Travel                        10000 non-null  object
 4   Department                             10000 non-null  object
 5   Distance_From_Home                     10000 non-null  int64 
 6   Education                              10000 non-null  object
 7   Environment_Satisfaction               10000 non-null  int64 
 8   Gender                                 10000 non-null  object
 9   Salary                                 10000 non-null  int64 
 10  Job_Involvement                        10000 non-null  int64 
 11  Job_Level       

In [8]:
df.describe()

,Employee_ID,Age,Attrition,Distance_From_Home,Environment_Satisfaction,Salary,Job_Involvement,Job_Level,Job_Satisfaction,Number_of_Companies_Worked_previously,Salary_Hike_in_percent,Total_working_years_experience,Work_life_balance,No_of_years_worked_at_current_company,No_of_years_in_current_role,Years_since_last_promotion
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,40.494900,0.224700,25.445100,2.987800,114492.519000,2.986300,4.53120,2.991900,5.009800,50.148600,9.978800,3.003600,4.978000,5.003500,3.985400
std,2886.89568,13.162298,0.417405,14.390431,1.414939,49009.843725,1.422995,2.28564,1.427178,3.166371,29.154528,6.110686,1.412865,3.177849,3.148187,2.587944
min,1.00000,18.000000,0.000000,1.000000,1.000000,30078.000000,1.000000,1.00000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,2500.75000,29.000000,0.000000,13.000000,2.000000,72124.000000,2.000000,3.00000,2.000000,2.000000,25.000000,5.000000,2.000000,2.000000,2.000000,2.000000
50%,5000.50000,40.000000,0.000000,25.000000,3.000000,114499.000000,3.000000,5.00000,3.000000,5.000000,51.000000,10.000000,3.000000,5.000000,5.000000,4.000000
75%,7500.25000,52.000000,0.000000,38.000000,4.000000,157213.000000,4.000000,7.00000,4.000000,8.000000,75.000000,15.000000,4.000000,8.000000,8.000000,6.000000
max,10000.00000,63.000000,1.000000,50.000000,5.000000,199989.000000,5.000000,8.00000,5.000000,10.000000,100.000000,20.000000,5.000000,10.000000,10.000000,8.000000


In [9]:
X = df.drop(columns=['Attrition','Employee_ID'])
y = df[['Attrition']]

In [10]:
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

In [11]:
X[num_cols].skew()

Age                                      0.002882
Distance_From_Home                      -0.006268
Environment_Satisfaction                 0.006098
Salary                                   0.010765
Job_Involvement                          0.021009
Job_Level                               -0.015252
Job_Satisfaction                         0.007223
Number_of_Companies_Worked_previously   -0.013950
Salary_Hike_in_percent                  -0.004544
Total_working_years_experience          -0.004531
Work_life_balance                       -0.001049
No_of_years_worked_at_current_company    0.010026
No_of_years_in_current_role              0.002085
Years_since_last_promotion               0.006840
dtype: float64

In [12]:
X.head()

,Age,Business_Travel,Department,Distance_From_Home,Education,Environment_Satisfaction,Gender,Salary,Job_Involvement,Job_Level,Job_Role,Job_Satisfaction,Marital_Status,Number_of_Companies_Worked_previously,Overtime,Salary_Hike_in_percent,Total_working_years_experience,Work_life_balance,No_of_years_worked_at_current_company,No_of_years_in_current_role,Years_since_last_promotion
0,56,Travel Rarely,Network Administration,24,Graduation,5,Male,186630,2,6,Manager,4,Married,7,Yes,85,9,1,7,2,4
1,46,Travel Rarely,Software Development,45,Degree,5,Male,78553,4,5,Help Desk,2,Divorced,3,No,22,18,4,1,2,0
2,32,No Travel,Data Science,30,Master's,3,Female,175003,1,4,HR,2,Single,2,Yes,100,0,4,5,5,8
3,60,Travel Frequently,Data Science,42,Graduation,2,Female,100795,2,8,Help Desk,3,Married,5,No,72,17,2,1,7,3
4,25,No Travel,Network Administration,26,Graduation,4,Female,75256,1,2,Director,4,Single,9,No,21,12,5,7,2,4


In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

numerical_transformer = Pipeline(steps=[
    ('imputation', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputation', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

In [14]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
], remainder='passthrough')

In [15]:
from sklearn.linear_model import LogisticRegression

lg_bal = LogisticRegression(max_iter=2000, 
                            class_weight='balanced', 
                            random_state=42,
                            solver='saga')

lg_unbal = LogisticRegression(max_iter=2000, random_state=42, solver='saga')

dt_bal = DecisionTreeClassifier(class_weight='balanced', random_state=42)
dt_unbal = DecisionTreeClassifier(random_state=42)

rf_bal = RandomForestClassifier(n_estimators=1000, class_weight='balanced', random_state=42, 
                                criterion='log_loss', max_depth=8, min_samples_split=30, 
                                max_leaf_nodes = 30, max_samples = 0.8,
                                min_samples_leaf = 10)

rf_unbal = RandomForestClassifier(n_estimators=1000, random_state=42, criterion='log_loss', 
                                  max_depth=8, min_samples_split=30, 
                                  max_leaf_nodes = 30, max_samples = 0.8,
                                  min_samples_leaf = 10)

cat_bal = CatBoostClassifier(iterations=1000, learning_rate=0.1, auto_class_weights='SqrtBalanced', depth=5, random_state=42, 
                             cat_features=cat_cols, verbose=False)
cat_unbal = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=5, random_state=42, cat_features=cat_cols, verbose=False)

In [16]:
lg_bal_pipe = Pipeline(steps=[
    ('preprocessing', preprocessor), 
    ('lg_bal', lg_bal)
])

lg_unbal_pipe = Pipeline(steps=[
    ('preprocessing', preprocessor), 
    ('lg_unbal', lg_unbal)
])

dt_bal_pipe = Pipeline(steps=[
    ('preprocessing', preprocessor), 
    ('dt_bal', dt_bal)
])

dt_unbal_pipe = Pipeline(steps=[
    ('preprocessing', preprocessor), 
    ('dt_unbal', dt_unbal)
])

rf_bal_pipe = Pipeline(steps=[
    ('preprocessing', preprocessor), 
    ('rf_bal', rf_bal)
])

rf_unbal_pipe = Pipeline(steps=[
    ('preprocessing', preprocessor), 
    ('rf_unbal', rf_unbal)
])

In [17]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_lg_bal = cross_val_score(lg_bal_pipe, X, y.values.ravel(), cv=skf, n_jobs=1, scoring='roc_auc')
cv_lg_unbal = cross_val_score(lg_unbal_pipe, X, y.values.ravel(), cv=skf, n_jobs=1, scoring='roc_auc')

In [18]:
cv_dt_bal = cross_val_score(dt_bal_pipe, X, y.values.ravel(), cv=skf, n_jobs=1, scoring='roc_auc')
cv_dt_unbal = cross_val_score(dt_unbal_pipe, X, y.values.ravel(), cv=skf, n_jobs=1, scoring='roc_auc')

In [19]:
cv_rf_bal = cross_val_score(rf_bal_pipe, X, y.values.ravel(), cv=skf, n_jobs=1, scoring='roc_auc')
cv_rf_unbal = cross_val_score(rf_unbal_pipe, X, y.values.ravel(), cv=skf, n_jobs=1, scoring='roc_auc')

In [20]:
cv_lg_bal.mean(), cv_lg_unbal.mean()

(0.7535427338267192, 0.754022753319932)

In [21]:
cv_dt_bal.mean(), cv_dt_unbal.mean()

(0.7052860802962929, 0.6998304237187473)

In [22]:
cv_rf_bal.mean(), cv_rf_unbal.mean()

(0.8253888005737411, 0.826383826365021)

In [23]:
bal_scores = []
unbal_scores = []

for train_idx, val_idx in skf.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    
    cat_bal.fit(X_train, y_train, cat_features=cat_cols)
    cat_unbal.fit(X_train, y_train, cat_features=cat_cols)
    
    bal_preds = cat_bal.predict_proba(X_val)[:, 1]
    unbal_preds = cat_unbal.predict_proba(X_val)[:, 1]
    
    bal_scores.append(roc_auc_score(y_val, bal_preds))
    unbal_scores.append(roc_auc_score(y_val, unbal_preds))

print("Mean AUC for balanced data:", sum(bal_scores)/len(bal_scores))
print("Mean AUC for unbalanced data:", sum(unbal_scores)/len(unbal_scores))

Mean AUC for balanced data: 0.8201170306622052
Mean AUC for unbalanced data: 0.822139497498244


In [24]:
from sklearn.model_selection import cross_val_predict

pred_lg_bal = cross_val_predict(lg_bal_pipe, X, y.values.ravel(), cv=skf, n_jobs=1, method='predict_proba')[:, 1]
pred_lg_unbal = cross_val_predict(lg_unbal_pipe, X, y.values.ravel(), cv=skf, n_jobs=1, method='predict_proba')[:, 1]

print(roc_auc_score(y, pred_lg_bal))
print(roc_auc_score(y, pred_lg_unbal))

0.7534015142996171
0.7539270871559488


In [25]:
print('Metrics for Linear model (Logistic Regression)')
y_pred_bal_lg = (pred_lg_bal >= 0.45).astype(int)
y_pred_unbal_lg = (pred_lg_unbal >= 0.45).astype(int)

print('Classification report for imbalanced data\n', classification_report(y, y_pred_unbal_lg))
print('Confusion matrix for imbalanced data\n', confusion_matrix(y, y_pred_unbal_lg))
print('Classification report for balanced data\n', classification_report(y, y_pred_bal_lg))
print('Confusion matrix for balanced data\n', confusion_matrix(y, y_pred_bal_lg))

Metrics for Linear model (Logistic Regression)
Classification report for imbalanced data
               precision    recall  f1-score   support

           0       0.83      0.94      0.88      7753
           1       0.61      0.33      0.42      2247

    accuracy                           0.80     10000
   macro avg       0.72      0.63      0.65     10000
weighted avg       0.78      0.80      0.78     10000

Confusion matrix for imbalanced data
 [[7279  474]
 [1513  734]]
Classification report for balanced data
               precision    recall  f1-score   support

           0       0.89      0.61      0.72      7753
           1       0.36      0.75      0.48      2247

    accuracy                           0.64     10000
   macro avg       0.62      0.68      0.60     10000
weighted avg       0.77      0.64      0.67     10000

Confusion matrix for balanced data
 [[4719 3034]
 [ 565 1682]]


In [26]:
pred_dt_bal = cross_val_predict(dt_bal_pipe, X, y.values.ravel(), cv=skf, n_jobs=1, method='predict_proba')[:, 1]
pred_dt_unbal = cross_val_predict(dt_unbal_pipe, X, y.values.ravel(), cv=skf, n_jobs=1, method='predict_proba')[:, 1]

print(roc_auc_score(y, pred_dt_bal))
print(roc_auc_score(y, pred_dt_unbal))

0.7052928274861057
0.6998337235809374


In [27]:
print('Metrics for Tree model (Decision Tree)')
y_pred_bal_dt = (pred_dt_bal >= 0.45).astype(int)
y_pred_unbal_dt = (pred_dt_unbal >= 0.45).astype(int)

print('Classification report for imbalanced data\n', classification_report(y, y_pred_unbal_dt))
print('Confusion matrix for imbalanced data\n', confusion_matrix(y, y_pred_unbal_dt))
print('Classification report for balanced data\n', classification_report(y, y_pred_bal_dt))
print('Confusion matrix for balanced data\n', confusion_matrix(y, y_pred_bal_dt))

Metrics for Tree model (Decision Tree)
Classification report for imbalanced data
               precision    recall  f1-score   support

           0       0.87      0.84      0.86      7753
           1       0.51      0.55      0.53      2247

    accuracy                           0.78     10000
   macro avg       0.69      0.70      0.69     10000
weighted avg       0.79      0.78      0.78     10000

Confusion matrix for imbalanced data
 [[6549 1204]
 [1000 1247]]
Classification report for balanced data
               precision    recall  f1-score   support

           0       0.87      0.86      0.86      7753
           1       0.53      0.55      0.54      2247

    accuracy                           0.79     10000
   macro avg       0.70      0.71      0.70     10000
weighted avg       0.79      0.79      0.79     10000

Confusion matrix for balanced data
 [[6644 1109]
 [1003 1244]]


In [28]:
pred_rf_bal = cross_val_predict(rf_bal_pipe, X, y.values.ravel(), cv=skf, n_jobs=1, method='predict_proba')[:, 1]
pred_rf_unbal = cross_val_predict(rf_unbal_pipe, X, y.values.ravel(), cv=skf, n_jobs=1, method='predict_proba')[:, 1]

print(roc_auc_score(y, pred_rf_bal))
print(roc_auc_score(y, pred_rf_unbal))

0.8249744804988419
0.8261050706013222


In [29]:
print('Metrics for Ensemble model (Random Forest)')
y_pred_bal_rf = (pred_rf_bal >= 0.45).astype(int)
y_pred_unbal_rf = (pred_rf_unbal >= 0.45).astype(int)

print('Classification report for imbalanced data\n', classification_report(y, y_pred_unbal_rf))
print('Confusion matrix for imbalanced data\n', confusion_matrix(y, y_pred_unbal_rf))
print('Classification report for balanced data\n', classification_report(y, y_pred_bal_rf))
print('Confusion matrix for balanced data\n', confusion_matrix(y, y_pred_bal_rf))

Metrics for Ensemble model (Random Forest)
Classification report for imbalanced data
               precision    recall  f1-score   support

           0       0.80      1.00      0.89      7753
           1       0.99      0.15      0.25      2247

    accuracy                           0.81     10000
   macro avg       0.90      0.57      0.57     10000
weighted avg       0.84      0.81      0.75     10000

Confusion matrix for imbalanced data
 [[7751    2]
 [1920  327]]
Classification report for balanced data
               precision    recall  f1-score   support

           0       0.93      0.62      0.74      7753
           1       0.39      0.83      0.53      2247

    accuracy                           0.67     10000
   macro avg       0.66      0.72      0.63     10000
weighted avg       0.80      0.67      0.69     10000

Confusion matrix for balanced data
 [[4784 2969]
 [ 380 1867]]


In [30]:
from catboost import Pool, cv

train_pool = Pool(
    data=X,
    label=y,
    cat_features=cat_cols
)

params = {
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "iterations": 1000,
    "depth": 4,
    "learning_rate": 0.1,
    "random_seed": 42,
    "verbose": False,
    "auto_class_weights": "Balanced"
}

cv_results = cv(
    pool=train_pool,
    params=params,
    fold_count=5,
    stratified=True,
    shuffle=True
)

cv_results.tail()

Training on fold [0/5]

bestTest = 0.8546772691
bestIteration = 35

Training on fold [1/5]

bestTest = 0.8263056093
bestIteration = 85

Training on fold [2/5]

bestTest = 0.8344268157
bestIteration = 57

Training on fold [3/5]

bestTest = 0.8452043969
bestIteration = 49

Training on fold [4/5]

bestTest = 0.8459472663
bestIteration = 67



,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
995,995,0.826550,0.006941,0.517024,0.011613,0.250440,0.005632
996,996,0.826554,0.006939,0.517061,0.011594,0.250317,0.005664
997,997,0.826529,0.007003,0.517157,0.011669,0.250179,0.005663
998,998,0.826420,0.006976,0.517304,0.011612,0.250037,0.005696
999,999,0.826427,0.007023,0.517323,0.011669,0.249906,0.005702


In [31]:
cat_bal.fit(X, y.values.ravel(), cat_features=cat_cols)
cat_unbal.fit(X, y.values.ravel(), cat_features=cat_cols)

In [32]:
pred_cat_bal = cat_bal.predict_proba(X)[:, 1]
pred_cat_unbal = cat_unbal.predict_proba(X)[:, 1]

In [33]:
print("ROC-AUC score for imbalanced data:", roc_auc_score(y, pred_cat_unbal))
print("ROC-AUC score for balanced data:", roc_auc_score(y, pred_cat_bal))

ROC-AUC score for imbalanced data: 0.9833484788551925
ROC-AUC score for balanced data: 0.9895363013504799


In [34]:
print('Metrics for catboost classifier')
ypred_cat_unbal = (pred_cat_unbal >= 0.45).astype(int)
ypred_cat_bal = (pred_cat_bal >= 0.45).astype(int)
print('Classification report for imbalanced data\n', classification_report(y, ypred_cat_unbal))
print('Confusion matrix for imbalanced data\n', confusion_matrix(y, ypred_cat_unbal))
print('Classification report for balanced data\n', classification_report(y, ypred_cat_bal))
print('Confusion matrix for balanced data\n', confusion_matrix(y, ypred_cat_bal))

Metrics for catboost classifier
Classification report for imbalanced data
               precision    recall  f1-score   support

           0       0.93      1.00      0.96      7753
           1       0.99      0.72      0.83      2247

    accuracy                           0.94     10000
   macro avg       0.96      0.86      0.90     10000
weighted avg       0.94      0.94      0.93     10000

Confusion matrix for imbalanced data
 [[7731   22]
 [ 626 1621]]
Classification report for balanced data
               precision    recall  f1-score   support

           0       0.97      0.98      0.97      7753
           1       0.93      0.88      0.91      2247

    accuracy                           0.96     10000
   macro avg       0.95      0.93      0.94     10000
weighted avg       0.96      0.96      0.96     10000

Confusion matrix for balanced data
 [[7610  143]
 [ 270 1977]]


In [35]:
from sklearn.metrics import average_precision_score

print("PR-AUC score for imbalanced data:-", average_precision_score(y, pred_cat_unbal))
print("PR-AUC score for balanced data:-", average_precision_score(y, pred_cat_bal))

PR-AUC score for imbalanced data:- 0.9607620389989484
PR-AUC score for balanced data:- 0.9707797185854408


In [36]:
rf_bal_pipe.fit(X, y.values.ravel())
rf_unbal_pipe.fit(X, y.values.ravel())

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('rf_unbal', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different tran

In [37]:
prob_rf_bal = rf_bal_pipe.predict_proba(X)[:, 1]
prob_rf_unbal = rf_unbal_pipe.predict_proba(X)[:, 1]

In [38]:
print('roc_auc score for imbalanced data:-', roc_auc_score(y, prob_rf_unbal))
print('roc_auc score for balanced data:-', roc_auc_score(y, prob_rf_bal))
print('pr_auc score for imbalanced data:-', average_precision_score(y, prob_rf_unbal))
print('pr_auc score for balanced data:-', average_precision_score(y, prob_rf_bal))

roc_auc score for imbalanced data:- 0.8541209854249969
roc_auc score for balanced data:- 0.8570357449814423
pr_auc score for imbalanced data:- 0.7465544256201107
pr_auc score for balanced data:- 0.7412147575157834


In [39]:
print('Metrics for Random Forest classifier')
ypred_rf_unbal = (prob_rf_unbal >= 0.45).astype(int)
ypred_rf_bal = (prob_rf_bal >= 0.45).astype(int)
print('Classification report for imbalanced data\n', classification_report(y, ypred_rf_unbal))
print('Confusion matrix for imbalanced data\n', confusion_matrix(y, ypred_rf_unbal))
print('Classification report for balanced data\n', classification_report(y, ypred_rf_bal))
print('Confusion matrix for balanced data\n', confusion_matrix(y, ypred_rf_bal))

Metrics for Random Forest classifier
Classification report for imbalanced data
               precision    recall  f1-score   support

           0       0.80      1.00      0.89      7753
           1       1.00      0.15      0.27      2247

    accuracy                           0.81     10000
   macro avg       0.90      0.58      0.58     10000
weighted avg       0.85      0.81      0.75     10000

Confusion matrix for imbalanced data
 [[7753    0]
 [1903  344]]
Classification report for balanced data
               precision    recall  f1-score   support

           0       0.94      0.62      0.75      7753
           1       0.40      0.87      0.55      2247

    accuracy                           0.68     10000
   macro avg       0.67      0.74      0.65     10000
weighted avg       0.82      0.68      0.70     10000

Confusion matrix for balanced data
 [[4795 2958]
 [ 291 1956]]


In [40]:
final_dt_bal = DecisionTreeClassifier(class_weight='balanced', random_state=42, criterion="log_loss",
                                      max_depth=8, max_leaf_nodes=40, min_samples_leaf=10, min_samples_split=30)
                                      
final_dt_unbal = DecisionTreeClassifier(random_state=42, criterion="log_loss",
                                      max_depth=8, max_leaf_nodes=40, min_samples_leaf=10, min_samples_split=30)

final_dt_bal_pipe = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('dt_bal', final_dt_bal)
])

final_dt_unbal_pipe = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('dt_bal', final_dt_unbal)
])

In [42]:
final_dt_bal_pipe.fit(X, y.values.ravel())
final_dt_unbal_pipe.fit(X, y.values.ravel())

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('dt_bal', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transf

In [43]:
prob_dt_bal = final_dt_bal_pipe.predict_proba(X)[:, 1]
prob_dt_unbal = final_dt_unbal_pipe.predict_proba(X)[:, 1]

In [44]:
print(roc_auc_score(y, prob_dt_bal))
print(roc_auc_score(y, prob_dt_unbal))
print(average_precision_score(y, prob_dt_bal))
print(average_precision_score(y, prob_dt_unbal))

0.8447643420514942
0.8456904087718088
0.7184830185901026
0.7222930434547788


In [45]:
pred_dt_bal = (prob_dt_bal >= 0.45).astype(int)
pred_dt_unbal = (prob_dt_unbal >= 0.45).astype(int)
print(classification_report(y, pred_dt_unbal))
print(classification_report(y, pred_dt_bal))

              precision    recall  f1-score   support

           0       0.86      0.99      0.92      7753
           1       0.95      0.46      0.62      2247

    accuracy                           0.87     10000
   macro avg       0.90      0.72      0.77     10000
weighted avg       0.88      0.87      0.85     10000

              precision    recall  f1-score   support

           0       0.92      0.71      0.80      7753
           1       0.44      0.78      0.56      2247

    accuracy                           0.72     10000
   macro avg       0.68      0.74      0.68     10000
weighted avg       0.81      0.72      0.74     10000



In [46]:
print(confusion_matrix(y, pred_dt_unbal))
print(confusion_matrix(y, pred_dt_bal))

[[7694   59]
 [1223 1024]]
[[5481 2272]
 [ 495 1752]]


In [47]:
import joblib
joblib.dump(cat_bal, 'C:/Users/aksha/employee_attrition_project/cat_attrition.joblib')
joblib.dump(rf_bal_pipe, 'C:/Users/aksha/employee_attrition_project/rf_attrition.joblib')
joblib.dump(dt_bal_pipe, 'C:/Users/aksha/employee_attrition_project/dt_attrition.joblib')

['C:/Users/aksha/employee_attrition_project/dt_attrition.joblib']

In [48]:
joblib.dump(lg_bal_pipe, 'C:/Users/aksha/employee_attrition_project/lg_attrition.joblib')

['C:/Users/aksha/employee_attrition_project/lg_attrition.joblib']